In [ ]:
#About Jacknife:

#ImportError: cannot import name 'bootstrap' from 'scipy.stats
#pip install -U scipy

# Practice 2. Sample size. Robust regressions.

## Data

Again, the TIMSS 2019 data. We will use the data for United Arab Emirates.
Data is already prepared as file **timss_all_data.csv** (preparation is the same as in the previous practice)


Columns:

+ math score - maths score (Mathematics Achievement, points scored by the student for tasks in mathematics)
+ sex - boy\girl (0\1)
+ education - Highest education of the parents
+ interest - interest in maths (from not interested at all to love to maths )
+ teaching - Teaching assessment (from he/she help to not clear, no help, not interesting to teacher’s explanations are clear and interesting things learned)
+ success - success in maths (from no success to success)
+ importance - attitude to maths, its importance for life (from not important at all to very important)

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('timss_uae.csv')

In [ ]:
data = data[['math_score', 'sex', 'education', 'interest', 'teaching', 'success', 'importance']]

In [ ]:
data.head()

Let's make 3 more datasets of different size.

In [ ]:
data100 = data.sample(n=100, random_state=1)
data700 = data.sample(n=700, random_state=1)
data5000 = data.sample(n=5000, random_state=1)

## Testing Hypotheses

In [ ]:
from scipy import stats

### Simple tests

1. There is no differences in math score between girls and boys.

full data

In [ ]:
# Welch t-test
res = stats.ttest_ind(data[data['sex'] == 0]['math_score'], 
                      data[data['sex'] == 1]['math_score'], 
                      equal_var=False)
print(res)

100 observations

In [ ]:
res = stats.ttest_ind(data100[data100['sex'] == 0]['math_score'], 
                      data100[data100['sex'] == 1]['math_score'], 
                      equal_var=False)
print(res)

700 observations

In [ ]:
res = stats.ttest_ind(data700[data700['sex'] == 0]['math_score'], 
                      data700[data700['sex'] == 1]['math_score'], 
                      equal_var=False)
print(res)

5000 observations

In [ ]:
res = stats.ttest_ind(data5000[data5000['sex'] == 0]['math_score'], 
                      data5000[data5000['sex'] == 1]['math_score'], 
                      equal_var=False)
print(res)

### Linear Regression

Target variable - math score

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

In [ ]:
import statsmodels.formula.api as smf

m_full = smf.ols(formula="math_score ~ C(sex) + interest + teaching + success + importance + C(education, Treatment('Secondary or lower'))", 
                 data=data).fit()
m_full.summary()

In [ ]:
m_100 = smf.ols(formula="math_score ~ C(sex) + interest + teaching + success + importance + C(education, Treatment('Secondary or lower'))", 
                 data=data100).fit()
m_100.summary()

In [ ]:
m_700 = smf.ols(formula="math_score ~ C(sex) + interest + teaching + success + importance + C(education, Treatment('Secondary or lower'))", 
                 data=data700).fit()
m_700.summary()

In [ ]:
m_5000 = smf.ols(formula="math_score ~ C(sex) + interest + teaching + success + importance + C(education, Treatment('Secondary or lower'))", 
                 data=data5000).fit()
m_5000.summary()

## Robust measures

Robust measures of scale are methods that quantify the statistical dispersion in a sample of numerical data while *resisting outliers*. 

The most common such robust statistics are **the interquartile range (IQR)** and **the median absolute deviation (MAD)**. These are contrasted with conventional or non-robust measures of scale, such as sample variance or standard deviation, which are greatly influenced by outliers.



**The interquartile range (IQR)**: the difference between the 75th percentile and the 25th percentile of a sample

In [ ]:
q3, q1 = np.percentile(data['math_score'], [75 ,25])
iqr = q3 - q1

In [ ]:
data['math_score'].hist()

In [ ]:
iqr

**Robust Regressions**

https://www.statsmodels.org/devel/rlm.html

Huber, RANSAC, TheilSen


Huber regression - Square error for inliers, linear for outliers.

In [ ]:
# Huber Regression

import statsmodels.api as sm
Y = data['math_score']
X = sm.add_constant(pd.concat([data[['sex', 'interest', 
                                     'teaching', 'success', 'importance']], 
                               pd.get_dummies(data['education'])[["Don't Know", "Post-Secondary", 
                                                                  "University or Higher"]]], axis=1))

rlm = sm.RLM(Y, X, M=sm.robust.norms.HuberT()).fit()
print(rlm.summary())

## Multimodal data

read the following link about some methods for working with multimodal features:

https://www.kaggle.com/code/iamleonie/handling-multimodal-distributions-fe-techniques